In [39]:
import requests
from requests_oauthlib import OAuth1
import cnfg
import pandas as pd
import datetime
import pickle
import time

In [2]:
config = cnfg.load(".twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

unit testing

In [17]:
screen_name = 'realdonaldtrump'
search_params = {'screen_name': [screen_name],
                 'count': 200
                 }

In [18]:
response = requests.get('https://api.twitter.com/1.1/statuses/user_timeline.json',
                        params = search_params,
                        auth = oauth)

tweets = response.json()

In [27]:
df = pd.DataFrame(tweets)

In [29]:
df.shape

(200, 29)

In [19]:
for tweet in tweets:
    print tweet['text']
    print

Melania will be interviewed by @morningmika on @Morning_Joe now (8:30 A.M.). ENJOY!

THANK YOU NEVADA!
#Trump2016 #MakeAmericaGreatAgain
@Snapchat! 
Username: realdonaldtrump 
https://t.co/e0HX1zekX8 https://t.co/uVBGpXzICo

THANK YOU NEVADA! WE WILL MAKE AMERICA SAFE &amp; GREAT AGAIN! 
https://t.co/ClAnLIg0AD https://t.co/zVvyVCIaD4

Make sure you get on the Trump line and are not mislead by the Cruz people. They are bad! BE CAREFUL.

RT @TrumpNV: #NVcaucus locator -&gt; https://t.co/WIzgQcbvOB

Exclusive interview w/ my wife @MELANIATRUMP- tomorrow morning @ 8amE on @Morning_Joe w/ @morningmika @MSNBC. Enjoy! https://t.co/FQSEKWxnwj

GO VOTE FROM NOW TO 8:30 P.M. NEVADA. I WILL BE AT VARIOUS CAUCUS SITES. MAKE AMERICA GREAT AGAIN!

RT @EricTrump: Very proud of what my father has accomplished in the past 7 months -Wishing him amazing luck and success tonight! #NVcaucus …

My wife Melania will be on @Morning_Joe  tomorrow morning at 8:00. Interviewed by @morningmika - Enjoy!

Thank yo

**Create recursive function to get all tweets based on max_id.**

In [6]:
# define screen names for candidates
candidates = {'clinton' : 'hillaryclinton', 
              'donald' : 'realdonaldtrump', 
              'rubio' : 'marcorubio', 
              'sanders' : 'berniesanders', 
              'cruz' : 'tedcruz'}

In [167]:
max_calls = 150
max_tweets = 3500

In [162]:
### can use this if possible to get > 3200 tweets
# def get_tweets(screen_name, call = 1, max_id = None, tweets = []):
#     if len(tweets) == max_tweets:
#         return tweets 
#     if call > 180: 
#         print 'sleeping'
#         time.sleep(60*15)
#         call = 1 
#     try:
#         print call
#         search_params = {'screen_name': [screen_name], 
#                          'count': 200, 
#                          'max_id': max_id}
#         json = requests.get('https://api.twitter.com/1.1/statuses/user_timeline.json',
#                         params = search_params,
#                         auth = oauth).json()
#         last_id = json[-1]['id']
#         return tweets + get_tweets(screen_name, call + 1, last_id, json)
#     except:
#         return tweets

def get_tweets(screen_name, call = 1, max_id = None, tweets = []):
    if call > max_calls:
        return tweets
    print call
    search_params = {'screen_name': [screen_name], 
                     'count': 200, 
                     'max_id': max_id}
    json = requests.get('https://api.twitter.com/1.1/statuses/user_timeline.json',
                    params = search_params,
                    auth = oauth).json()
    last_id = json[-1]['id']
    try: # catches the first instance 
        if tweets[-1]['id'] == last_id: # check if we ran out of tweets 
            return tweets
        else:
            return tweets + get_tweets(screen_name, call + 1, last_id, json)
    except:
        return tweets + get_tweets(screen_name, call + 1, last_id, json)

In [194]:
cruz_tweets = get_tweets(candidates['cruz'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [187]:
rubio_tweets = get_tweets(candidates['rubio'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [168]:
hillary_tweets = get_tweets(candidates['clinton'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [184]:
donald_tweets = get_tweets('realdonaldtrump')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [173]:
bernie_tweets = get_tweets(candidates['sanders'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [198]:
kanye_tweets = get_tweets('kanyewest')

1
2
3
4
5


In [202]:
for tweet in kanye_tweets:
    print tweet['text']

I'm going to steal Demna from Balenciaga
You won't always agree with me but I'm gon always be me. I'm woke!
The world needs a guy like me. The world needs somebody to not be scared and tell his truth.
I love being a voice of freedom when so many people are scared to speak up.
I represent what people can't say.
New album coming this summer
I'm not even gon lie to you. I love me so much right now.
And yes I've talked to Adidas and we gon hook y'all up with free Yeezys and Adidas.  All positive vibes.
That is one of the reasons I respect the paps so much.
Respect to everyone working hard to keep a roof over their kids
What is your definition of true freedom? There is no true freedom without economic freedom.
I just had to start with shoes so that I could be economically empowered enough to have a voice.
You guys will see my heart. You guys will feel my heart. We will make a difference.
It's a beautiful time.  I love my friends. I love my family.  I love people. The human race.
Free from b

In [197]:
# not sure if this works
all_tweets = []
for c in candidates:
    all_tweets.append(get_tweets(candidates[c]))

sanders
donald
clinton
rubio
cruz


**Pickle**

In [23]:
def pickle_file(tweets):
    with open('data/%s.pkl' %tweets, 'w') as picklefile:
        pickle.dump(tweets, picklefile)

In [188]:
with open('tweets/donald_tweets.pkl', 'w') as picklefile:
    pickle.dump(donald_tweets, picklefile)

In [189]:
with open('tweets/hillary_tweets.pkl', 'w') as picklefile:
    pickle.dump(hillary_tweets, picklefile)

In [190]:
with open('tweets/bernie_tweets.pkl', 'w') as picklefile:
    pickle.dump(bernie_tweets, picklefile)

In [191]:
with open('tweets/rubio_tweets.pkl', 'w') as picklefile:
    pickle.dump(rubio_tweets, picklefile)

In [195]:
with open('tweets/cruz_tweets.pkl', 'w') as picklefile:
    pickle.dump(cruz_tweets, picklefile)

In [199]:
with open('tweets/kanye_tweets.pkl', 'w') as picklefile:
    pickle.dump(kanye_tweets, picklefile)

**Get current rate limit usage.**

In [75]:
def get_remaining_calls():
    """Get remaining calls for statuses/user_timeline requests."""
    usage = requests.get('https://api.twitter.com/1.1/application/rate_limit_status.json?resources=statuses', 
                     auth = oauth).json()
    detail = usage['resources']['statuses']['/statuses/user_timeline']
    next_reset = datetime.datetime.fromtimestamp(detail['reset']).strftime('%Y-%m-%d %H:%M:%S')
    return detail['remaining'], next_reset

In [192]:
remaining_calls, next_reset = get_remaining_calls()

In [193]:
print remaining_calls
print next_reset

162
2016-02-24 15:07:09


#### Import Tweets into MongoDB

In [23]:
import json
from pymongo import MongoClient


client = MongoClient()
db = client.legislation
# Our collection: 
tweets = db.new_tweets

In [24]:
for tweet in results:
    data={}
    data['tweet']=tweet.text.encode('utf-8') 
    data['datetime']=tweet.created_at
    tweets.insert(data)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [57]:
tweets.find_one()

{u'_id': ObjectId('56cc3bdd24b5a555771f36d8'),
 u'datetime': datetime.datetime(2016, 2, 23, 5, 1, 46),
 u'tweet': u'Troy was in vintage super bowl highlights. Serving water.'}